In [18]:
# packages
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from claspy.segmentation import BinaryClaSPSegmentation

import ruptures as rpt

import stumpy
from aeon.segmentation import find_dominant_window_sizes

from aeon.segmentation import GreedyGaussianSegmenter

from aeon.segmentation import InformationGainSegmenter

from aeon.anomaly_detection import STRAY

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer,mean_squared_error
from ruptures.metrics import precision_recall
import matplotlib.pyplot as plt
#from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles
import seaborn as sns

sns.set_theme()
sns.set_color_codes()

from claspy.tests.evaluation import f_measure,covering

In [19]:
def IgnoreZone(idx,cpraw):
    cp = cpraw.tolist()
    if idx == 0:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 3944.7118557910376+100 and cp[i] < 5911.693516853054-100 or cp[i] > 12845.0+100:
                cp.pop(i)
                
        
    elif idx == 1:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 2874.607407407407+100 and cp[i] < 4016.935849056604-100:
                cp.pop(i)

    elif idx == 16:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 180.03455207940698+100 and cp[i] < 1227.051137077522-100 or cp[i] > 5865.505591154668+100:
                cp.pop(i)

    elif idx == 17:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 138.33224102186853+100 and cp[i] < 3677.231833076974-100:
                cp.pop(i)


    elif idx == 19:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 8187.634803581529+100:
                cp.pop(i)


    else:
        print("error IgnoreZone()")
    return np.array(cp)

In [20]:
def delnear(arr,range):
    i = 0
    while i < len(arr) - 1:
        # Iniziamo con il primo elemento di un potenziale gruppo
        gruppo_inizio = i
        gruppo_fine = i

        # Cerca gli elementi che fanno parte dello stesso gruppo
        while gruppo_fine < len(arr) - 1 and arr[gruppo_fine + 1] - arr[gruppo_fine] < range:
            gruppo_fine += 1

        # Se esiste un gruppo di più elementi
        if gruppo_fine > gruppo_inizio:
            # Se la distanza tra l'inizio e la fine è minore di 50, elimina l'elemento maggiore (gruppo_fine)
            if arr[gruppo_fine] - arr[gruppo_inizio] < range:
                arr = np.delete(arr, gruppo_fine)
            
            # Elimina tutti gli elementi interni al gruppo
            arr = np.concatenate((arr[:gruppo_inizio + 1], arr[gruppo_fine:]))

        # Procedi con il prossimo gruppo
        i = gruppo_inizio + 1

    return arr

In [21]:
def f1scoremargin(ground_truth, predictions, tolerance):
    """
    Calcola l'F1 score con una finestra di tolleranza sui change points.
    
    :param ground_truth: Lista o array di change points reali
    :param predictions: Lista o array di change points predetti
    :param tolerance: La tolleranza temporale (numero di unità temporali)
    :return: precision, recall, f1-score
    """
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    
    # Vettori per tracciare quali punti sono stati già associati
    matched_ground_truth = np.zeros(len(ground_truth), dtype=bool)
    matched_predictions = np.zeros(len(predictions), dtype=bool)

    mgt={key: False for key in ground_truth}
    mcp={key: False for key in predictions}
    #print(f'gt:{len(ground_truth)} - cp:{len(predictions)}')
    # True Positives (TP)
    tp = 0
    for i, gt_point in enumerate(ground_truth):
        for j, pred_point in enumerate(predictions):
            if not matched_predictions[j] and abs(gt_point - pred_point) <= tolerance:
                tp += 1
                matched_ground_truth[i] = True
                matched_predictions[j] = True

                mgt[gt_point] = True
                mcp[pred_point] = True
                break
            
    
    # False Positives (FP) - predizioni non corrispondenti a nessun ground truth entro la tolleranza
    fp = np.sum(~matched_predictions)
    
    # False Negatives (FN) - punti del ground truth non corrispondenti a nessuna predizione entro la tolleranza
    fn = np.sum(~matched_ground_truth)
    #print(f'tp:{tp} - fp:{fp} - fn:{fn}')
    #print(mgt)
    #print(mcp)
    # Calcolo di precision, recall e F1-score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1, tp, fp, fn

Extracted features. Use the index of this list to use with iloc[]
<ol start="0">
  <li>Kinetic Global</li>
  <li>Kinetic Chest</li>
  <li>Directness Head</li>
  <li>Density</li>
  <li>left wrist ke</li>
  <li>right wrist ke</li>
  <li>left ankle ke</li>
  <li>right ankle ke</li>
  <li>head ke</li>
  <li>crouch density</li>
  <li>left leg density</li>
  <li>right leg density</li>
  <li>left hand density</li>
  <li>right hand density</li>
  <li>hand density</li>
  <li>arto inferiore</li>
  <li>gamba</li>
  <li>coscia</li>
  <li>coscia dx</li>
  <li>coscia sx</li>
  <li>gamba sx</li>
  <li>gamba dx</li>
  <li>braccio sx</li>
  <li>braccio dx</li>
  <li>avambraccio sx</li>
  <li>avambraccio dx</li>
  <li>ARIEL speed magnitude</li>
  <li>ARIEL speed X component</li>
  <li>ARIEL speed Y component</li>
  <li>ARIEL speed Z component</li>
  <li>ARIEL acceleration magnitude</li>
  <li>ARIEL acceleration X component</li>
  <li>ARIEL acceleration Y component</li>
  <li>ARIEL acceleration Z component</li>
  <li>ARIEL jerk magnitude</li>
  <li>ARIEL jerk X component</li>
  <li>ARIEL jerk Y component</li>
  <li>ARIEL jerk Z component</li>
  <li>STRN speed magnitude</li>
  <li>STRN speed X component</li>
  <li>STRN speed Y component</li>
  <li>STRN speed Z component</li>
  <li>STRN acceleration magnitude</li>
  <li>STRN acceleration X component</li>
  <li>STRN acceleration Y component</li>
  <li>STRN accelerationZ component</li>
  <li>STRN jerk magnitude</li>
  <li>STRN jerk X component</li>
  <li>STRN jerk Y component</li>
  <li>STRN jerk Z component</li>
  <li>RHEL speed magnitude</li>
  <li>RHEL speed X component</li>
  <li>RHEL speed Y component</li>
  <li>RHEL speed Z component</li>
  <li>RHEL acceleration magnitude</li>
  <li>RHEL acceleration X component</li>
  <li>RHEL acceleration Y component</li>
  <li>RHEL acceleration Z component</li>
  <li>RHEL jerk magnitude</li>
  <li>RHEL jerk X component</li>
  <li>RHEL jerk Y component</li>
  <li>RHEL jerk Z component</li>
  <li>LHEL speed magnitude</li>
  <li>LHEL speed X component</li>
  <li>LHEL speed Y component</li>
  <li>LHEL speed Z component</li>
  <li>LHEL acceleration magnitude</li>
  <li>LHEL acceleration X component</li>
  <li>LHEL acceleration Y component</li>
  <li>LHEL acceleration Z component</li>
  <li>LHEL jerk magnitude</li>
  <li>LHEL jerk X component</li>
  <li>LHEL jerk Y component</li>
  <li>LHEL jerk Z component</li>
  <li>RPLM speed magnitude</li>
  <li>RPLM speed X component</li>
  <li>RPLM speed Y component</li>
  <li>RPLM speed Z component</li>
  <li>RPLM acceleration magnitude</li>
  <li>RPLM acceleration X component</li>
  <li>RPLM acceleration Y component</li>
  <li>RPLM acceleration Z component</li>
  <li>RPLM jerk magnitude</li>
  <li>RPLM jerk X component</li>
  <li>RPLM jerk Y component</li>
  <li>RPLM jerk Z component</li>
  <li>LPLM speed magnitude</li>
  <li>LPLM speed X component</li>
  <li>LPLM speed Y component</li>
  <li>LPLM speed Z component</li>
  <li>LPLM acceleration magnitude</li>
  <li>LPLM acceleration X component</li>
  <li>LPLM acceleration Y component</li>
  <li>LPLM acceleration Z component</li>
  <li>LPLM jerk magnitude</li>
  <li>LPLM jerk X component</li>
  <li>LPLM jerk Y component</li>
  <li>LPLM jerk Z component</li>




</ol>

In [22]:
# list of features. To access its name or its value while using iloc
features_name=[
    "kinetic_global",
    "kinetic_chest",
    "directness_head",
    "density",
    "left_wrist_ke",
    "right_wrist_ke",
    "left_ankle_ke",
    "right_ankle_ke",
    "head_ke",
    "crouch_density",
    "left_leg_density",
    "right_leg_density",
    "left_hand_density",
    "right_hand_density",
    "hand_density",
    "arto_inferiore",
    "gamba",
    "coscia",
    "coscia_dx",
    "coscia_sx",
    "gamba_sx",
    "gamba_dx",
    "braccio_sx",
    "braccio_dx",
    "avambraccio_sx",
    "avambraccio_dx",
    "ARIEL_speed_magnitude",
    "ARIEL_speed_X_component",
    "ARIEL_speed_Y_component",
    "ARIEL_speed_Z_component",
    "ARIEL_acceleration_magnitude",
    "ARIEL_acceleration_X_component",
    "ARIEL_acceleration_Y_component",
    "ARIEL_acceleration_Z_component",
    "ARIEL_jerk_magnitude",
    "ARIEL_jerk_X_component",
    "ARIEL_jerk_Y_component",
    "ARIEL_jerk_Z_component",
    "STRN_speed_magnitude",
    "STRN_speed_X_component",
    "STRN_speed_Y_component",
    "STRN_speed_Z_component",
    "STRN_acceleration_magnitude",
    "STRN_acceleration_X_component",
    "STRN_acceleration_Y_component",
    "STRN_acceleration_Z_component",
    "STRN_jerk_magnitude",
    "STRN_jerk_X_component",
    "STRN_jerk_Y_component",
    "STRN_jerk_Z_component",
    "RHEL_speed_magnitude",
    "RHEL_speed_X_component",
    "RHEL_speed_Y_component",
    "RHEL_speed_Z_component",
    "RHEL_acceleration_magnitude",
    "RHEL_acceleration_X_component",
    "RHEL_acceleration_Y_component",
    "RHEL_acceleration_Z_component",
    "RHEL_jerk_magnitude",
    "RHEL_jerk_X_component",
    "RHEL_jerk_Y_component",
    "RHEL_jerk_Z_component",
    "LHEL_speed_magnitude",
    "LHEL_speed_X_component",
    "LHEL_speed_Y_component",
    "LHEL_speed_Z_component",
    "LHEL_acceleration_magnitude",
    "LHEL_acceleration_X_component",
    "LHEL_acceleration_Y_component",
    "LHEL_acceleration_Z_component",
    "LHEL_jerk_magnitude",
    "LHEL_jerk_X_component",
    "LHEL_jerk_Y_component",
    "LHEL_jerk_Z_component",
    "RPLM_speed_magnitude",
    "RPLM_speed_X_component",
    "RPLM_speed_Y_component",
    "RPLM_speed_Z_component",
    "RPLM_acceleration_magnitude",
    "RPLM_acceleration_X_component",
    "RPLM_acceleration_Y_component",
    "RPLM_acceleration_Z_component",
    "RPLM_jerk_magnitude",
    "RPLM_jerk_X_component",
    "RPLM_jerk_Y_component",
    "RPLM_jerk_Z_component",
    "LPLM_speed_magnitude",
    "LPLM_speed_X_component",
    "LPLM_speed_Y_component",
    "LPLM_speed_Z_component",
    "LPLM_acceleration_magnitude",
    "LPLM_acceleration_X_component",
    "LPLM_acceleration_Y_component",
    "LPLM_acceleration_Z_component",
    "LPLM_jerk_magnitude",
    "LPLM_jerk_X_component",
    "LPLM_jerk_Y_component",
    "LPLM_jerk_Z_component",
]

In [23]:
# Questi sono tutte le features estratte
# kineticglobal,kineticchest,directnesshead,density,leftwristke,rightwristke,leftankleke,rightankleke,headke,posturaltension

# Queste sono le feature che utilizzeremo
# kineticglobal, density, leftwirstke, rightwristke, leftankleke, rightankleke, headke, posturaltension
# che corrispondono
# 2,4,5,6,7,8,9,10,11
# Questa funzione legge il file di input e restituisce un dataframe con i dati preprocessati
# ATTENZIONE
# il motivo per cui 4 corrisponde a density, mentre in realtà è il 3 è perchè df.drop prima non cancellava
# perciò il iloc[:,0] corrispondeva alla colonna time che era rimasta
# iloc considera le colonne come array
def ReadAndPreProcess(inputDataRaw):
    # lettura
    df=pd.read_csv(inputDataRaw,sep=' ', header=None).interpolate()
    
    df=df.drop(0, axis=1)
    # i dati usciti da eyesweb hanno le feature inverse, qui le faccio rimettere nel ordine giusto
    df = df.iloc[:, ::-1]
  
    # ce un bug che se faccio reverse delle colonne mi da overflow int troppo grande per conversione
    #df["som"]=df.sum(axis=1)
    ## preprocessing

    # dealing NaN values
    #-serafino ha usato forward fill, backward fill, linear interpolation
    #-ricordo che serafino aveva gia utilizzato sta cosa sui dati grezzi non sulle feature ma sui dati prefeature percio dovrebbe essere gia apposto

    # downsampling
    #-sono a 100ms, non sò se devo scendere a 50ms. da decidere
    #-non lo faccio xk non mi interessa se va piu lento, guarda su notion per ulteriori informazioni

    # low pass filter
    #-Skogstad and colleagues (2013) e https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units
    #-implementare dopo

    # remove outliers
    #-utilizzare hampel filter

    # stretch
    #-forse devo stretcharlo come ho fatto precedentemente
    #-anche se nel codice precedente ho stretchato solo il groundtruth

    # ritorno un oggetto dataframe dopo che è stato lavorato, ottenendo un prodotto lavorato
    # faccio un arrotondamento perchè il mio dataset ha troppi valori decimali e vado in overflow
    return df#.round(2)


In [24]:
# questa funzione ritorna un dataframe del groundtruth che viene usato specificatamente per visualizzare il gt
# è soggetto a un preprocessing dei dati siccome l'ultimo groundtruth è dove termina il ts del gt
# di conseguenza per farlo corrispondere, bisogna stretcharlo
# ma ricordo di aver rifatti i dati nuovi per generare un groundtruth a fine ts, da controllare cosi che non serve stretcharlo?
def LoadingGroundTruth(df,gtraw):
    gt=pd.read_csv(gtraw,sep=' ', header=None)
    gt=gt.iloc[:,0].values
    #stretching dei dati se necessario per farlo corrispondere alla ts dei dati
    stretch_gt = np.array([])
    for idx,i in enumerate(gt):
        relpos = len(df)*i/gt[-1]
        stretch_gt = np.append(stretch_gt,relpos)

    # eliminiamo l'ultimo elemento cbhe è stato annotato solo per delimitare la lunghezza della gt simile alla ts
    return stretch_gt[:-1]

In [34]:
def GetClasp(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for idx,i in enumerate(range(0,len(features_name))):
      
        ts=df.iloc[:,i]
        clasp = BinaryClaSPSegmentation(**kwargs)
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp

In [35]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClaspOld(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for idx,i in enumerate([1,3,4,5,6,7,8]):
        
        ts=df.iloc[:,i]
        clasp = BinaryClaSPSegmentation(**kwargs)
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        
        
    return result, eachresult, eachclasp
        
    


In [36]:
# calcola i vari scores dati il groundtruth e il prediction
# puo salvare il risultato su file per evitare di perderli
# prende come parametro nome del groundtruth, groundtruth, nome della timeseries e il prediction
def Evaluate(modelName,gtName, gt, tsName, cp, df, margin,msg, nomeFile):
    # creo dei array di lunghezza come la ts cosi possono fare il confronto
    # sia per il gt che per il pd
  
    cpnump = np.array(cp)
    gtnump = np.array(gt)

    cp_long = np.zeros(len(df)+1)
    cp_long[cpnump.astype(int)]=1

    gt_long = np.zeros(len(df)+1)
    gt_long[gtnump.astype(int)]=1

    # calcolo lo score 
    print(f'f1margin: {f1scoremargin(gt.astype(int),cp.astype(int),margin)}')
    precision,recall,f1,tp,fp,fn=f1scoremargin(gt.astype(int),cp.astype(int),margin)

    return precision,recall,f1
     #scrivo su file il risultato

    """
    f = open("outputFile/ClaSPallresult", "a")
    f.write(nomeFile+"\n")
    f.write(msg+"\n")
    f.write("precision:"+str(precision)+" recall:"+str(recall)+" f1:"+str(f1)+" \n")
    f.write("\n")
    f.close()
    """
    

In [37]:
def EvaluateEvery(eachcp,gt,margin):
    result=[]
    resultratio=[]
    
    for idx,cp in enumerate(eachcp):
        precision,recall,f1,tp,fp,fn=f1scoremargin(gt.astype(int),eachcp[idx].astype(int),margin)
        result.append(f1)
        resultratio.append(str(tp)+"/"+str(fp)+"/"+str(fn))
    return result,resultratio

In [38]:
def Plotclasp(eachclasp,gt,margin,eachcp,nomeFile):
    
    for idx,clasp in enumerate(eachclasp):
        clasp.plot(gt_cps=gt.astype(int), heading=f'f1margin: {f1scoremargin(gt.astype(int),eachcp[idx].astype(int),margin)} {nomeFile}', ts_name="suss", file_path="segmentation_example.png")


    
        for idx2,j in enumerate(gt.astype(int)):
            plt.fill_betweenx(np.array([0, 1]), j-margin, j+margin, color='green', alpha=0.3)


In [39]:
def PlotResult(df,gt,cp, nomeFile, margin):
    #da testare quando ho piu valori
    #clasp.plot(gt_cps=gt.astype(int), heading="Segmentation of different umpire cricket signals", ts_name="ACC", file_path="segmentation_example.png")

    plt.figure(figsize=(18,9))
    plt.plot(np.arange(len(df["som"].values)),df["som"].values,'blue',linewidth=0.5)
    for idx2,i in enumerate(gt.astype(int)):

            plt.axvline(x = i, color = 'green',linewidth=1) 
            
    for j in cp.tolist():
        plt.axvline(x = j, color = 'red',linewidth=1,linestyle="-.") 

    for idx2,k in enumerate(gt.astype(int)):
            plt.fill_betweenx(np.array([0, 1]), k-margin, k+margin, color='green', alpha=0.3)
    plt.xlabel(f'{nomeFile} {f1scoremargin(gt.astype(int),cp.astype(int),margin)}')



In [40]:
timeseries=[
      "in\cora1_in.txt",
      "in\cora4_05_in.txt",
      "in\cora4_08_in.txt",
      "in\cora5_in.txt",
      "in\cora14_in.txt",
      "in\marianne7_in.txt",
      "in\marianne8_in.txt",
      "in\marianne10_in.txt",
      "in\marianne18_in.txt",
      "in\marianne19_in.txt",
      "in\marianne24_in.txt",
      "in\marianne26_in.txt",
      "in\marianne41_in.txt",
      "in\marianne42_in.txt",
      "in\marianne43_in.txt",
      "in\marianne47_in.txt",
      "in\marianne48_in.txt",
      "in\muriel18_in.txt",
      "in\muriel26_in.txt",
      "in\muriel27_in.txt",
      "in\muriel30_in.txt"
      ]
groundtruth=[
      "gt\cora_gt_2019-08-08_t001_video01.txt",
      "gt\cora_gt_2019-05-22_t004_video01.txt",
      "gt\cora_gt_2019-08-08_t004_video01.txt",
      "gt\cora5_gt.txt",
      "gt\cora_gt_2019-08-08_t014_video01.txt",
      "gt\marianne_gt_2016-03-22_t007_video01.txt",
      "gt\marianne_gt_2016-03-22_t008_video01.txt",
      "gt\marianne_gt_2016-03-22_t010_video01.txt",
      "gt\marianne_gt_2016-03-22_t018_video01.txt",
      "gt\marianne_gt_2016-03-22_t019_video01.txt",
      "gt\marianne_gt_2016-03-22_t024_video01.txt",
      "gt\marianne_gt_2016-03-22_t026_video01.txt",
      "gt\marianne_gt_2016-03-22_t041_video01.txt",
      "gt\marianne_gt_2016-03-22_t042_video01.txt",
      "gt\marianne_gt_2016-03-22_t043_video01.txt",
      "gt\marianne_gt_2016-03-22_t047_video01.txt",
      "gt\marianne_gt_2016-03-22_t048_video01.txt",
      "gt\muriel_gt_2016-03-21_t018_video01.txt",
      "gt\muriel_gt_2016-03-21_t026_video01.txt",
      "gt\muriel_gt_2016-03-21_t027_video01.txt",
      "gt\muriel_gt_2016-03-23_t030_video01.txt"
      ]


In [41]:
import pandas as pd
test = pd.DataFrame(columns =['Name', 'Code', 'Age', 'Weight'])
test = pd.concat([test, pd.DataFrame([[1,2,3,4]], columns=test.columns)], ignore_index=True)
test = pd.concat([test, pd.DataFrame([[1,2,3,4]], columns=test.columns)], ignore_index=True)
test



,Name,Code,Age,Weight
0,1,2,3,4
1,1,2,3,4


In [ ]:
timeseries_old=[
      "in_old\cora1_input.txt",
      "in\cora4_05_in.txt",
      "in_old\cora4_08_input.txt",
      "in_old\cora5_input.txt",
      "in_old\cora14_input.txt",
      "in_old\marianne7_input.txt",
      "in_old\marianne8_input.txt",
      "in_old\marianne10_input.txt",
      "in_old\marianne18_input.txt",
      "in_old\marianne19_input.txt",
      "in_old\marianne24_input.txt",
      "in_old\marianne26_input.txt",
      "in_old\marianne41_input.txt",
      "in_old\marianne42_input.txt",
      "in_old\marianne43_input.txt",
      "in_old\marianne47_input.txt",
      "in_old\marianne48_input.txt",
      "in_old\muriel18_input.txt",
      "in_old\muriel26_input.txt",
      "in_old\muriel27_input.txt",
      "in_old\muriel30_input.txt"
      ]

In [ ]:
# colonna dei nomi dei file
cleaned_array = [s[3:-7] for s in timeseries]

# excel file per output f1score con feature classiche
outdf = pd.DataFrame({"name":cleaned_array})


#suss, euclidean, 4
outeval=[]
for i in range(len(timeseries_old)):
    
    df=ReadAndPreProcess(timeseries_old[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size=100, distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries_old[i],cp,df,100,"suss euclidean 4",timeseries_old[i])
    outeval.append(f1)
outdf["suss_euclidean_4"] = outeval

#fft, euclidean, 4
outeval=[]
for i in range(len(timeseries_old)):
    
    df=ReadAndPreProcess(timeseries_old[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size="fft", distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries_old[i],cp,df,100,"fft euclidean 4",timeseries_old[i])
    outeval.append(f1)
outdf["fft_euclidean_4"] = outeval

#acf, euclidean, 4
outeval=[]
for i in range(len(timeseries_old)):
    
    df=ReadAndPreProcess(timeseries_old[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size="acf", distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries_old[i],cp,df,100,"acf euclidean 4",timeseries_old[i])
    outeval.append(f1)
outdf["acf_euclidean_4"] = outeval
outdf.to_excel("outputFile/outputALL_oldIn.xlsx")

# colonna dei nomi dei file
cleaned_array = [s[3:-7] for s in timeseries]

# excel file per output f1score con feature classiche
outdf = pd.DataFrame({"name":cleaned_array})


#suss, euclidean, 4
outeval=[]
for i in range(len(timeseries)):
    
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size=100, distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss euclidean 4",timeseries[i])
    outeval.append(f1)
outdf["suss_euclidean_4"] = outeval

#fft, euclidean, 4
outeval=[]
for i in range(len(timeseries)):
    
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size="fft", distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"fft euclidean 4",timeseries[i])
    outeval.append(f1)
outdf["fft_euclidean_4"] = outeval

#acf, euclidean, 4
outeval=[]
for i in range(len(timeseries)):
    
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClaspOld(df,gt,0,window_size="acf", distance="euclidean_distance", excl_radius=4)
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=IgnoreZone(i,cp)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"acf euclidean 4",timeseries[i])
    outeval.append(f1)
outdf["acf_euclidean_4"] = outeval



#f1scores
outdf2 = pd.DataFrame(columns = features_name)#['kineticglobal', 'density','leftwirstke', 'rightwristke', 'leftankleke', 'rightankleke', 'headke'])

#ratio prediction
outdf3 = pd.DataFrame(columns = features_name)

#suss, euclidean, 4
outeval=[]
for i in range(len(timeseries)):
    print(timeseries[i])
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,window_size="suss", distance="euclidean_distance", excl_radius=4)
    for idx,j in enumerate(eachcp):
        eachcp[idx] = IgnoreZone(i,j)
    f1_list,ratio_list=EvaluateEvery(eachcp,gt,100)
    outdf2 = pd.concat([outdf2, pd.DataFrame([f1_list], columns=outdf2.columns)], ignore_index=True)
    outdf3 = pd.concat([outdf3, pd.DataFrame([ratio_list], columns=outdf3.columns)], ignore_index=True)
outdf2["name"]=cleaned_array
outdf3["name"]=cleaned_array

outdf.to_excel("outputFile/outputALL.xlsx")
outdf2.to_excel("outputFile/outputALLsingles.xlsx")
outdf3.to_excel("outputFile/outputALLsinglesRATIO.xlsx")


    
df=ReadAndPreProcess(timeseries[3])
gt=LoadingGroundTruth(df,groundtruth[3])
cp,eachcp,clasp=GetClasp(df,gt,0,window_size="suss", distance="euclidean_distance", excl_radius=4,n_jobs =8)
for idx,j in enumerate(eachcp):
    eachcp[idx] = IgnoreZone(3,j)
f1_list,ratio_list=EvaluateEvery(eachcp,gt,100)